### Try to export the model weights from PyTorch ---> ONNX ---> TensorFlow (Didn't work well...)

In [ ]:
import tractolearn.models.track_ae_cnn1d_incr_feat_strided_conv_fc_upsamp_reflect_pad_pytorch as AE_model

In [ ]:
weights_path = "/home/teitxe/data/tractolearn_data/"
state_dict = torch.load(os.path.join(weights_path, "best_model_contrastive_tractoinferno_hcp.pt"), map_location=torch.device('cpu'))
net = AE_model.IncrFeatStridedConvFCUpsampReflectPadAE(32)
dummy_input = torch.randn(1, 3, 256)
net(dummy_input)[0][0][0:2]

#### Load the weights into the model and export them to ONNX

In [ ]:
import torch
net.load_state_dict(state_dict["state_dict"])
onnx_file = os.path.join(weights_path, "best_model_contrastive_tractoinferno_hcp.onnx")
torch.onnx.export(net, dummy_input, onnx_file, input_names=["input"], output_names=["output"], 
                  export_params=True)

#### Load the ONNX file

In [ ]:
import subprocess as sp
if not os.path.exists('/home/teitxe/data/tractolearn_data/tf_model'):
    container = 'docker run --rm -v /home/teitxe:/workdir -w /workdir docker.io/pinto0309/onnx2tf:1.22.3 /bin/bash -c "'
    tf_model_path = '/workdir/data/tractolearn_data/tf_model'
    onnx_model_path = '/workdir/data/tractolearn_data/best_model_contrastive_tractoinferno_hcp.onnx'
    command = f'mkdir -p {tf_model_path} && onnx2tf -i {onnx_model_path} -o {tf_model_path}"'
    print(container + command)
    sp.run(container + command, shell=True, check=True)

#### Try to load the model into TF

In [ ]:
local_tf_model_path = '/home/teitxe/data/tractolearn_data/tf_model'
model = tf.saved_model.load(local_tf_model_path)


print((model(input_streamline))[0][0])

In [ ]:
output_torch[0][0]

In [ ]:
outputs = []
for streamline in streamlines:
    outputs.append(model(streamline.reshape(1, 256, 3))[0])

In [ ]:
# Copy the header of MNI152
mni152 = nib.load("/home/teitxe/data/tractolearn_data/mni_masked.nii.gz")
tractogram = nib.streamlines.Tractogram(outputs, affine_to_rasmm=mni152.affine)
nib.streamlines.save(tractogram, "test.trk", header=mni152.header)